# Defi-IA 2022 <a href="https://www.kaggle.com/c/defi-ia-2022/overview"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png" style="max-width: 70px; display: inline" alt="Python"/></a>
-------------------------------------------------------------

**Predict the accumulated daily rainfall on ground stations**  
*Quentin Douzery, Alexia Ghozland, Dario Moed*

## <font color = 'royalblue'>1 Imports</font>

In [1]:
##Packages
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [3]:
##Données
coords_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Other/stations_coordinates.csv' #Perso
#coords_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Other/stations_coordinates.csv' #INSA
#coords_fname  = '/kaggle/input/defi-ia-2022-test/Other/Other/stations_coordinates.csv'

xtrainS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/X_station_train.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/X_station_train.csv'

ytrain_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/Y_train.csv' #Perso
#ytrain_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/Y_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/Y_train.csv'

xtestS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/X_station_test.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA

bl_obsT_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/Baselines/Baseline_observation_test.csv' #Perso

## <font color = 'royalblue'>2 Récupération des données</font>

In [4]:
##Baseline observations test
bl_obsT = pd.read_csv(bl_obsT_path,infer_datetime_format=True)
display(bl_obsT)
print('Missing Data :', bl_obsT.isna().sum().sum(), '\n', 'Shape is :', bl_obsT.shape)

,Id,Prediction
0,14066001_149,1.4
1,14126001_149,1.0
2,14137001_149,0.4
3,14216001_149,0.8
4,14296001_149,3.6
...,...,...
85135,86137003_293,0.2
85136,86165005_293,0.2
85137,86273001_293,0.2
85138,91200002_293,0.2


Missing Data : 0 
 Shape is : (85140, 2)


In [5]:
##X station test
xtestS = pd.read_csv(xtestS_path,infer_datetime_format=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8


Missing Data : 3914287 
 Shape is : (2304802, 8)


## <font color = 'royalblue'>3 Pré-processing</font>

In [7]:
def Get_StationDay (id):
    return id.split('_')[0] + "_" + id.split('_')[1]

In [8]:
xtestS = xtestS.assign(StationDay=pd.Series(np.zeros(xtestS.shape[0])).values)
xtestS.loc[:,"StationDay"] = xtestS.loc[:,"Id"].apply(Get_StationDay)
display(xtestS)

,dd,hu,td,t,ff,precip,month,Id,StationDay
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,14047002_277
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,14047002_277
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,14047002_277
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,14047002_277
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,14047002_277
...,...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19,95690001_176
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20,95690001_176
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21,95690001_176
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22,95690001_176


In [16]:
My_bl_obsT = xtestS[{"StationDay","precip"}]
My_bl_obsT = My_bl_obsT.groupby("StationDay", as_index=False).agg(pd.Series.sum, min_count = 24)
#My_bl_obsT = My_bl_obsT('StationDay').agg(pd.Series.sum, min_count = 24)
display(My_bl_obsT)

,StationDay,precip
0,14047002_100,9.9
1,14047002_223,0.8
2,14047002_236,7.0
3,14047002_256,4.0
4,14047002_277,NaN
...,...,...
96050,95690001_95,0.0
96051,95690001_96,1.8
96052,95690001_97,0.0
96053,95690001_98,8.0


## <font color = 'royalblue'>4 Prédictions</font>

In [24]:
My_bl_obsT.rename(columns={"StationDay":"Id", "precip":"Prediction"}, inplace=True)
display(My_bl_obsT)

,Id,Prediction
0,14047002_100,9.9
1,14047002_223,0.8
2,14047002_236,7.0
3,14047002_256,4.0
4,14047002_277,NaN
...,...,...
96050,95690001_95,0.0
96051,95690001_96,1.8
96052,95690001_97,0.0
96053,95690001_98,8.0


In [42]:
First_predictions = My_bl_obsT.loc[My_bl_obsT['Id'].isin(bl_obsT["Id"])]
First_predictions.reset_index(drop=True,inplace=True)
display(First_predictions)
print('Missing Data :', First_predictions.isna().sum().sum(), '\n', 'Shape is :', First_predictions.shape)

,Id,Prediction
0,14047002_100,9.9
1,14047002_223,0.8
2,14047002_236,7.0
3,14047002_256,4.0
4,14047002_281,0.0
...,...,...
85135,95690001_95,0.0
85136,95690001_96,1.8
85137,95690001_97,0.0
85138,95690001_98,8.0


Missing Data : 0 
 Shape is : (85140, 2)


## <font color = 'royalblue'>5 Post-processing</font>

In [43]:
First_predictions["Prediction"] = First_predictions["Prediction"] + 1
First_predictions = First_predictions.set_index('Id')
First_predictions = First_predictions.reindex(index=bl_obsT['Id'])
First_predictions = First_predictions.reset_index()
display(First_predictions)

<ipython-input-43-d3bf79d10dfb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  First_predictions["Prediction"] = First_predictions["Prediction"] + 1


,Id,Prediction
0,14066001_149,2.4
1,14126001_149,2.0
2,14137001_149,1.4
3,14216001_149,1.8
4,14296001_149,4.6
...,...,...
85135,86137003_293,1.2
85136,86165005_293,1.2
85137,86273001_293,1.2
85138,91200002_293,1.2


## <font color = 'royalblue'>6 Exportation</font>



In [45]:
output_file = "Submissions_1_baseline-bis.csv"
First_predictions.to_csv('/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Subimissions/' + output_file, index=False)

### Autres

In [6]:
#read the ground station data
def read_gs_data(fname):
    gs_data = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    gs_data = gs_data.sort_values(by=["number_sta","date"])
    return gs_data

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


In [13]:
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)

In [17]:
def Prediction_DayBefore (x, first_date, last_date):
    ##get the observation baseline
    #Base_obs = x[{"number_sta","date","precip"}]
    Base_obs = x[{"precip"}]
    #Base_obs.set_index('date',inplace = True)  

    ##compute the accumulated rainfall per day with nan management 
    ##if any NaN on the day, then the value is NaN (24 values per day)
    #Base_obs = Base_obs.groupby('number_sta').resample('D').agg(pd.Series.sum, min_count = 24)
    #Base_obs = Base_obs.reset_index(['date','number_sta'])
    #Base_obs['number_sta'] = Base_obs['number_sta'].astype('category') 

    ##Select the observations the day before 
    Base_obs['baseline_obs'] = Base_obs.groupby(['number_sta'])['precip'].shift(1)
    Base_obs = Base_obs.sort_values(by=["number_sta","date"])
    del Base_obs['precip']
    Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
    
    ##get the day indexes (to the final Id)
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    
    y = pd.merge(Base_obs,d_dates,how="right",on = ["date"])
    y = y[y['date']!=last_date]
    y['Id'] = y['number_sta'].astype(str) + '_' + \
                      y['day_index'].astype(str) 
    
    ##final post-processing
    del y['day_index']
    y = y.rename(columns={'precip':'Prediction'})
    
    return y